In [ ]:
import random
import time
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

def generate_boxes(n):
    """生成随机排列的盒子"""
    boxes = list(range(1, n + 1))
    random.shuffle(boxes)
    return boxes

def random_strategy(boxes, prisoner, k):
    """随机策略：囚犯随机打开k个盒子"""
    n = len(boxes)
    # 随机选择k个盒子
    selected_boxes = random.sample(range(n), k)
    # 检查是否找到自己的编号
    for box_idx in selected_boxes:
        if boxes[box_idx] == prisoner:
            return True
    return False

def cycle_strategy(boxes, prisoner, k):
    """循环策略：囚犯从自己编号的盒子开始追踪链条"""
    n = len(boxes)
    current_box = prisoner - 1  # 从自己编号的盒子开始
    steps = 0
    
    while steps < k:
        steps += 1
        # 打开盒子
        content = boxes[current_box]
        # 如果找到自己的编号
        if content == prisoner:
            return True
        # 否则跳转到下一个盒子
        current_box = content - 1  # 盒子编号从1开始，索引从0开始
    
    return False

def simulate_single_trial(n, k, strategy):
    """单次试验模拟"""
    boxes = generate_boxes(n)
    
    for prisoner in range(1, n + 1):
        if strategy == "random":
            found = random_strategy(boxes, prisoner, k)
        else:  # "cycle" strategy
            found = cycle_strategy(boxes, prisoner, k)
        
        if not found:
            return False  # 任何囚犯失败则全体失败
    
    return True  # 所有囚犯都找到自己的编号

def get_cycle_lengths(boxes):
    """获取排列的循环长度分布"""
    n = len(boxes)
    visited = [False] * n
    cycle_lengths = []
    
    for i in range(n):
        if not visited[i]:
            cycle_len = 0
            current = i
            while not visited[current]:
                visited[current] = True
                cycle_len += 1
                current = boxes[current] - 1  # 盒子内容减1得到索引
            cycle_lengths.append(cycle_len)
    
    return cycle_lengths

def calculate_theoretical_success(n, k):
    """计算循环策略的理论成功率"""
    # 使用近似公式：1 - ln(2) * (1 - k/n) 当 n 很大时
    # 更精确的理论需要积分计算
    if k < n / 2:
        return 0.0
    
    # 简单近似
    return 1 - np.log(2) * (1 - k/n)

def main():
    """主函数：处理用户交互和模拟"""
    print("100囚犯抽签问题仿真")
    print("=" * 50)
    
    # 获取用户输入
    try:
        n = int(input("请输入囚犯数量N (默认100): ") or 100)
        k = int(input("请输入每人尝试次数K (默认50): ") or 50)
        trials = int(input("请输入模拟轮次T (默认10000): ") or 10000)
    except ValueError:
        print("输入无效，使用默认值: N=100, K=50, T=10000")
        n, k, trials = 100, 50, 10000
    
    if n < 1 or k < 1 or trials < 1:
        print("输入值必须为正整数，使用默认值")
        n, k, trials = 100, 50, 10000
    
    # 模拟随机策略
    print("\n模拟随机策略...")
    start_time = time.time()
    random_success = 0
    for _ in range(trials):
        if simulate_single_trial(n, k, "random"):
            random_success += 1
    random_success_rate = random_success / trials
    random_time = time.time() - start_time
    
    # 模拟循环策略
    print("模拟循环策略...")
    start_time = time.time()
    cycle_success = 0
    max_cycle_lengths = []  # 记录每次试验的最大循环长度
    
    for _ in range(trials):
        boxes = generate_boxes(n)
        cycle_lengths = get_cycle_lengths(boxes)
        max_cycle = max(cycle_lengths) if cycle_lengths else 0
        max_cycle_lengths.append(max_cycle)
        
        # 检查最大循环长度是否小于等于k
        if max_cycle <= k:
            cycle_success += 1
    
    cycle_success_rate = cycle_success / trials
    cycle_time = time.time() - start_time
    
    # 计算理论成功率
    theoretical_success = calculate_theoretical_success(n, k)
    
    # 输出结果
    print("\n结果统计:")
    print(f"囚犯数量(N): {n}, 尝试次数(K): {k}, 模拟轮次(T): {trials}")
    print(f"随机策略成功率: {random_success_rate:.8f} ({random_success}/{trials})")
    print(f"循环策略成功率: {cycle_success_rate:.6f} ({cycle_success}/{trials})")
    print(f"理论近似成功率: {theoretical_success:.6f}")
    print(f"随机策略模拟时间: {random_time:.2f}秒")
    print(f"循环策略模拟时间: {cycle_time:.2f}秒")
    
    # 分析循环长度分布
    print("\n循环策略分析:")
    success_lengths = [l for l in max_cycle_lengths if l <= k]
    failure_lengths = [l for l in max_cycle_lengths if l > k]
    
    print(f"平均最大循环长度: {np.mean(max_cycle_lengths):.2f}")
    print(f"成功试验的平均最大循环长度: {np.mean(success_lengths):.2f}" if success_lengths else "无成功试验")
    print(f"失败试验的平均最大循环长度: {np.mean(failure_lengths):.2f}" if failure_lengths else "无失败试验")
    
    # 绘制最大循环长度分布
    plt.figure(figsize=(12, 6))
    
    # 直方图
    plt.subplot(1, 2, 1)
    plt.hist(max_cycle_lengths, bins=range(1, n+2), alpha=0.7, color='skyblue', edgecolor='black')
    plt.axvline(x=k, color='r', linestyle='--', label=f'K={k} (尝试次数)')
    plt.title('最大循环长度分布')
    plt.xlabel('最大循环长度')
    plt.ylabel('频次')
    plt.legend()
    
    # 成功/失败对比
    plt.subplot(1, 2, 2)
    bins = np.linspace(1, n, 30)
    plt.hist(success_lengths, bins=bins, alpha=0.5, color='green', label='成功试验')
    plt.hist(failure_lengths, bins=bins, alpha=0.5, color='red', label='失败试验')
    plt.axvline(x=k, color='r', linestyle='--', label=f'K={k}')
    plt.title('成功与失败试验的循环长度对比')
    plt.xlabel('最大循环长度')
    plt.ylabel('频次')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 参数变化分析
    if n == 100 and k == 50:
        print("\n进行参数变化分析...")
        ratios = np.linspace(0.3, 0.7, 9)  # k/n 比例
        success_rates = []
        
        for ratio in ratios:
            k_val = int(n * ratio)
            success_count = 0
            
            # 模拟1000次试验
            for _ in range(1000):
                boxes = generate_boxes(n)
                cycle_lengths = get_cycle_lengths(boxes)
                max_cycle = max(cycle_lengths) if cycle_lengths else 0
                if max_cycle <= k_val:
                    success_count += 1
            
            success_rate = success_count / 1000
            success_rates.append(success_rate)
            print(f"K/N={ratio:.2f} (K={k_val}), 成功率: {success_rate:.4f}")
        
        # 绘制成功率随K/N变化曲线
        plt.figure(figsize=(10, 6))
        plt.plot(ratios, success_rates, 'o-', label='模拟成功率')
        
        # 添加理论曲线
        theoretical_rates = [calculate_theoretical_success(n, int(n * r)) for r in ratios]
        plt.plot(ratios, theoretical_rates, 'r--', label='理论近似')
        
        plt.axhline(y=0.3, color='gray', linestyle=':', alpha=0.5)
        plt.axvline(x=0.5, color='gray', linestyle=':', alpha=0.5)
        plt.title('循环策略成功率随K/N比例变化')
        plt.xlabel('K/N (尝试次数/囚犯数量)')
        plt.ylabel('成功率')
        plt.legend()
        plt.grid(True)
        plt.show()

if __name__ == "__main__":
    main()
